In [10]:
import mysql.connector
import pandas as pd

# MySQL 서버에 연결
conn = mysql.connector.connect(
    host='127.0.0.1',      # 호스트 이름
    user='root',       # MySQL 사용자 이름
    password='hj010701',   # MySQL 사용자 비밀번호
    database='data'  # 연결할 데이터베이스 이름
)

# 커서 생성
cursor = conn.cursor()

sql_query = "SELECT * FROM member_based_okr_assignments"
cursor.execute(sql_query)

result = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

member_based_okr_assignments = pd.DataFrame(result, columns=column_names)


sql_query = "SELECT * FROM okr_peer_30"
cursor.execute(sql_query)

result = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

okr_df = pd.DataFrame(result, columns=column_names)


sql_query = '''
SELECT *
FROM member_based_okr_assignments
JOIN okr_peer_30 
ON okr_peer_30.OKR_NUM IN (member_based_okr_assignments.project1, member_based_okr_assignments.project2, member_based_okr_assignments.project3);
'''
cursor.execute(sql_query)

result = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

member_okr = pd.DataFrame(result, columns=column_names)


,Member,project1,project2,project3,OKR_NUM,Objective,Key Result 1,Key Result 2,Key Result 3,Key Result 1 Score,...,3_frontend,4_frontend,5_frontend,6_frontend,1_backend,2_backend,3_backend,4_backend,5_backend,6_backend
0,1,OKR_1,OKR_4,OKR_19,OKR_4,Improve internal team communication tools,Increase message response rate by 15%,Reduce average time spent on internal updates ...,Achieve a 90% satisfaction rate with new team ...,82.1,...,4,2,5,1,5,5,4,4,5,4
1,1,OKR_1,OKR_4,OKR_19,OKR_19,Strengthen real-time collaboration features,Achieve a 25% increase in active use of collab...,Reduce latency for real-time updates by 30%,Increase user satisfaction with collaboration ...,59.4,...,5,5,5,4,3,4,4,2,2,2
2,1,OKR_1,OKR_4,OKR_19,OKR_1,Boost the user base through targeted marketing...,Increase sign-up conversion rates by 10%,Achieve a 20% increase in referral program par...,Reduce acquisition cost per user by 15%,62.4,...,5,4,4,3,3,2,1,4,1,2
3,2,OKR_7,OKR_10,OKR_18,OKR_7,Enhance user feedback incorporation in product...,Increase user feedback submission rates by 15%,Achieve a 20% reduction in time to implement u...,Improve user satisfaction with feedback-driven...,76.3,...,5,1,4,3,3,2,5,3,4,3
4,2,OKR_7,OKR_10,OKR_18,OKR_18,Boost mobile app downloads,Increase app store rating to 4.5,Achieve a 20% increase in organic downloads,Reduce crash rates by 50% on new versions,46.8,...,5,5,3,4,3,2,3,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,49,OKR_7,OKR_13,OKR_16,OKR_16,Increase customer satisfaction with new produc...,Achieve a 20% increase in positive feedback fo...,Reduce user-reported bugs by 30%,Increase user engagement on new features by 25%,55.3,...,4,3,4,1,2,2,3,2,2,3
146,49,OKR_7,OKR_13,OKR_16,OKR_13,Increase efficiency in cross-functional commun...,Reduce time spent in meetings by 25%,Increase documentation clarity by 30%,Achieve a 95% completion rate for action items...,34.2,...,2,4,4,5,4,4,4,3,5,5
147,50,OKR_20,OKR_21,OKR_22,OKR_22,Enhance the dashboard user experience,Increase user satisfaction with dashboard feat...,Reduce user-reported issues by 35%,Achieve a 20% increase in daily active users o...,68.6,...,5,4,4,2,4,5,5,3,1,4
148,50,OKR_20,OKR_21,OKR_22,OKR_21,Increase engagement with premium users,Achieve a 15% increase in retention rate for p...,Increase feedback submission rates by 20%,Reduce churn rate among premium users by 10%,54.7,...,5,4,4,5,3,2,3,4,4,5


In [7]:

import sys
import os
sys.path.append(os.path.abspath("../teambuilding"))
import pandas as pd
import numpy as np
import itertools

objectives = okr_df['Objective']
score=okr_df['Objective Score']

def generate_combinations_3d(data, num_parts=5):
    # Convert data to numpy array for easier manipulation
    data_values = data.values
    
    # Split data into equal parts
    part_size = len(data_values) // num_parts
    parts = [data_values[i * part_size:(i + 1) * part_size] for i in range(num_parts)]
    
    # Generate all possible combinations from the parts (row-wise combinations)
    combinations = list(itertools.product(*parts))
    
    # Convert combinations to a 3D numpy array
    combinations_3d = np.array(combinations)
    
    return combinations_3d


all_data_f = []

for i, n_okr in enumerate(objectives):
    # Step 2: cal.py의 calculate_weighted_scores 함수 사용하여 weighted_sums 계산
    weighted_sums = calculate_weighted_scores(n_okr)

    # Step 3: weighted_sums에서 두 번째 값을 추출
    weighted_values = [value[1] for value in weighted_sums]

    # Step 4: data의 첫 번째 열(0열)에 weighted_values 추가
    weighted_array = np.array(weighted_values)

   
    #
    if data.shape[0] == len(weighted_values):
        data.iloc[:, 0] = weighted_array  # pandas의 iloc 사용하여 첫 번째 열에 할당
        data['label']=score[i] #label 추가
    else:
        print(f"Objective {i+1}: 샘플 수가 일치하지 않습니다.")
    
    # Generate combinations based on the updated data
    data_3d = generate_combinations_3d(data.iloc[:, :], num_parts=5)

    # data_f 계산
    data_f = np.concatenate((data_3d[:,:,0:1], data_3d[:,:,4:]), axis=2)

    # data_f를 dim=0에서 쌓기 위해 리스트에 저장
    all_data_f.append(data_f)

# Step 5: dim=0에서 모든 data_f 연결
final_data_f = np.concatenate(all_data_f, axis=0)


print(final_data_f.shape)
print(final_data_f)

NameError: name 'calculate_weighted_scores' is not defined

In [2]:
okr_peer_30.head(1)

,OKR_NUM,Objective,Key Result 1,Key Result 2,Key Result 3,Key Result 1 Score,Key Result 2 Score,Key Result 3 Score,Objective Score,1_pm,...,3_frontend,4_frontend,5_frontend,6_frontend,1_backend,2_backend,3_backend,4_backend,5_backend,6_backend
0,OKR_1,Boost the user base through targeted marketing...,Increase sign-up conversion rates by 10%,Achieve a 20% increase in referral program par...,Reduce acquisition cost per user by 15%,62.4,37.9,79.8,60.03,3,...,5,4,4,3,3,2,1,4,1,2


In [4]:
member_based_okr_assignments.head(1)

,Member,project1,project2,project3
0,1,OKR_1,OKR_4,OKR_19
